In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
# Load the dataset with ds_info to get metadata
 (ds_train, ds_test), ds_info = tfds.load(
    'stanford_dogs',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True,
)

# Extract breed names using the metadata
breed_names = ds_info.features['label'].names

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...:   0%|          | 0/12000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stanford_dogs/0.2.0.incomplete6QFEWW/stanford_dogs-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/8580 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stanford_dogs/0.2.0.incomplete6QFEWW/stanford_dogs-test.tfrecord*...:   0%…

Dataset stanford_dogs downloaded and prepared to /root/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.


In [ ]:
## Part 2: Data Pre-processing ##
# Preprocessing function that will be applied to each element in the dataset.
def preprocess_image(image, label):
    image = tf.image.resize(image, [224, 224]) # Resize image to 224x224 to match model expected input
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Configure data augmentation to increase dataset variability
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

# Define a function to apply the augmentation to the images
def augment(image,label):
    image = data_augmentation(image)
    return image,label

# Apply preprocessing, augmentation, shuffle, batch, and prefetch.
ds_train = ds_train.map(preprocess_image).map(augment).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
## Part 3: Model Training ###

# Load MobileNetV2 as the base model, without the top layer, pre-trained on ImageNet
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the convolutional base.

# Create a new model and add additional layers on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(ds_info.features['label'].num_classes, activation='softmax') # Output layer with softmax activation to classify images
])

# Compile the model with the Adam optimizer and a suitable learning rate
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
checkpoint_cb = ModelCheckpoint("best_model.h5", save_best_only=True) # Save the best version of the model
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True) # Stop training when validation performance degrades
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=1) # Reduce learning rate when progress stalls

# Train the model using the training dataset, with validation data and callbacks
history = model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=10,
    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb]
)

9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/10
375/375 [==============================] - ETA: 0s - loss: 3.0929 - accuracy: 0.3246

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


375/375 [==============================] - 926s 2s/step - loss: 3.0929 - accuracy: 0.3246 - val_loss: 1.4008 - val_accuracy: 0.6776 - lr: 1.0000e-04
Epoch 2/10
375/375 [==============================] - 914s 2s/step - loss: 1.6614 - accuracy: 0.5588 - val_loss: 1.0066 - val_accuracy: 0.7232 - lr: 1.0000e-04
Epoch 3/10
375/375 [==============================] - 928s 2s/step - loss: 1.3903 - accuracy: 0.6120 - val_loss: 0.8820 - val_accuracy: 0.7435 - lr: 1.0000e-04
Epoch 4/10
375/375 [==============================] - 907s 2s/step - loss: 1.2684 - accuracy: 0.6402 - val_loss: 0.8421 - val_accuracy: 0.7507 - lr: 1.0000e-04
Epoch 5/10
375/375 [==============================] - 925s 2s/step - loss: 1.1814 - accuracy: 0.6599 - val_loss: 0.8017 - val_accuracy: 0.7610 - lr: 1.0000e-04
Epoch 6/10
375/375 [==============================] - 879s 2s/step - loss: 1.1178 - accuracy: 0.6762 - val_loss: 0.7876 - val_accuracy: 0.7618 - lr: 1.0000e-04
Epoch 7/10
375/375 [==============================]

In [ ]:
## Part 4: Fine-Tuning ##
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Re-compile the model for fine-tuning with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model on the training data, with fine-tuning adjustments
history_fine = model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=5,
    initial_epoch=history.epoch[-1],
    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb]
)

In [ ]:
import json

# Save breed names to a JSON file
with open('breed_names.json', 'w') as f:
    json.dump(breed_names, f)

In [ ]:
model.save('final_dog_breed_classifier.h5')

In [ ]:
from google.colab import files
files.download('final_dog_breed_classifier.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>